# Extract the toy model fields

### Constant potential

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 1         
N = 128 
tau_sim = 0.25    
DTS = 8

# Derived parameters
a = L/N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)
nplus = maxt//DTS

# Glasma fields
su_group = 'su3'
uv = 10.0
ir = 0.2
g2mu = 1.5


g = 2.0          		
mu = g2mu / g**2

ns = 50      

nevents = 1

In [2]:
import os

# Comment this, I need it for JYU GPUs to choose a specific GPU to run on
# os.environ["CUDA_VISIBLE_DEVICES"]="4"

os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

# Qhat module
import curraun.qhat as qhat

import pickle
from tqdm import tqdm

Using Numba
Using SU(3)
Using double precision


/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
tsteps = int(tau_sim/(L/N))

up = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
Ay = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
Az = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)

Asq = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)

for xplus in range (tsteps):
    for y in range(N):
        for z in range(N):
            A = np.full(8,-1j)
            Ay[xplus, y, z] = su.get_algebra_element(A)
            up[xplus, y, z] = su.unit()
            
            Asq[xplus, y, z]= su.mul(Ay[xplus, y, z], Ay[xplus, y, z])

/lustre/HQCD/carlos.lamas/curraun/SimpleTest/../curraun/su3.py:80: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../curraun/su3.py", line 398:
@myjit
def mul_s(g0, f):
^

  ms0 = mul_s(s1, algebra_factors[0])
/lustre/HQCD/carlos.lamas/curraun/SimpleTest/../curraun/su3.py:90: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../curraun/su3.py", line 369:
@myjit
def add(g0, g1):
^

  b0 = add(ms0, ms1)
/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/core/typed_passes.py:329: NumbaPerformanceWarning: 
The keyw

In [4]:
A_test = Ay[0, 0, 0].reshape(3, 3)
print(A_test)

[[ 0.78867513-0.j   0.5       -0.5j  0.5       -0.5j]
 [ 0.5       +0.5j -0.21132487+0.j   0.5       -0.5j]
 [ 0.5       +0.5j  0.5       +0.5j -0.57735027+0.j ]]


In [5]:
Asq_test = Asq[0, 0, 0].reshape(3, 3)
print(Asq_test)

[[1.62200847+0.j         0.78867513-0.28867513j 0.10566243-0.60566243j]
 [0.78867513+0.28867513j 1.0446582 +0.j         0.10566243+0.39433757j]
 [0.10566243+0.60566243j 0.10566243-0.39433757j 1.33333333+0.j        ]]


In [6]:
print(np.trace(Asq_test)/3)

(1.3333333333333333+0j)


In [7]:
# Go up one level from 'notebooks' to parent, then into 'simulations/tuomas_guess'
save_dir = os.path.join('..', 'SimpleTest', 'ConstantTest')
os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, 'up.npy'), up)
np.save(os.path.join(save_dir, 'Ay.npy'), Ay)
np.save(os.path.join(save_dir, 'Az.npy'), Az)

### Potential proportional to time

In [8]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 1         
N = 128 
tau_sim = 0.25    
DTS = 8

# Derived parameters
a = L/N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)
nplus = maxt//DTS

# Glasma fields
su_group = 'su3'
uv = 10.0
ir = 0.2
g2mu = 1.5


g = 2.0          		
mu = g2mu / g**2

ns = 50      

nevents = 1

In [9]:
import os

# Comment this, I need it for JYU GPUs to choose a specific GPU to run on
# os.environ["CUDA_VISIBLE_DEVICES"]="4"

os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

# Qhat module
import curraun.qhat as qhat

import pickle
from tqdm import tqdm

In [10]:
tsteps = int(tau_sim/(L/N))

up = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
Ay = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
Az = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)

Asq = np.zeros((tsteps, N, N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)

for xplus in range (tsteps):
    for y in range(N):
        for z in range(N):
            A = np.full(8,-1j*xplus*a/hbarc)
            Ay[xplus, y, z] = su.get_algebra_element(A)
            up[xplus, y, z] = su.unit()

In [11]:
# Go up one level from 'notebooks' to parent, then into 'simulations/tuomas_guess'
save_dir = os.path.join('..', 'SimpleTest', 'toy_model')
os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, 'up.npy'), up)
np.save(os.path.join(save_dir, 'Ay.npy'), Ay)
np.save(os.path.join(save_dir, 'Az.npy'), Az)